In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [9]:
file: str = 'urls.txt'
with open(file, 'r') as f:
    urls = f.readlines()
    webs = [url.strip() for url in urls]

In [10]:
# Define client headers
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

In [15]:
variables = ["Website", "Images", "Gifs", "Title Size", "Links", "Languages", "h1", "h2", "h3", "HTTPS", "Paragraphs", "Meta Description", "Meta Keywords", "JS Scripts", "Size KB", "Words", "Cookies"]
df = pd.DataFrame(columns=variables)
df["Website"] = webs

In [12]:
def get_numImages(soup):
    return len(soup.find_all('img'))

def get_numGifs(soup):
    return len(soup.find_all('gif'))

def is_https(url):
    return url.startswith('https')

def get_titleSize(soup):
    try:
        return len(soup.title.string)
    except:
        return None

def get_numLinks(soup):
    return len(soup.find_all('a'))

def get_languages(soup):
    try:
        return len(soup.html["lang"])
    except:
        return 0

def get_numH1(soup):
    return len(soup.find_all('h1'))

def get_numH2(soup):
    return len(soup.find_all('h2'))

def get_numH3(soup):
    return len(soup.find_all('h3'))

def get_Paragraphs(soup):
    return len(soup.find_all('p'))

def has_metadescription(soup):
    return len(soup.find_all('meta', attrs={'name': 'description'})) > 0

def has_metakeywords(soup):
    return len(soup.find_all('meta', attrs={'name': 'keywords'})) > 0

def javascript_size(soup):
    return sum([len(script) for script in soup.find_all('script')])

def num_words(soup):
    return len(soup.text.split())

In [16]:
for web in webs:
    print("Proccessing: ", web)
    try:
        response = requests.get(web, headers=headers)
    except:
        print("Error: Could not connect to ", web)
        continue
    if response.status_code != 200:
        print("Error: ", response.status_code, " for ", web)
        continue
    soup = BeautifulSoup(response.text, 'html.parser')
    df.loc[df["Website"] == web, "Images"] = get_numImages(soup)
    df.loc[df["Website"] == web, "Gifs"] = get_numGifs(soup)
    df.loc[df["Website"] == web, "HTTPS"] = is_https(web)
    df.loc[df["Website"] == web, "Title Size"] = get_titleSize(soup)
    df.loc[df["Website"] == web, "Links"] = get_numLinks(soup)
    df.loc[df["Website"] == web, "Languages"] = get_languages(soup)
    df.loc[df["Website"] == web, "h1"] = get_numH1(soup)
    df.loc[df["Website"] == web, "h2"] = get_numH2(soup)
    df.loc[df["Website"] == web, "h3"] = get_numH3(soup)
    df.loc[df["Website"] == web, "Paragraphs"] = get_Paragraphs(soup)
    df.loc[df["Website"] == web, "Meta Description"] = has_metadescription(soup)
    df.loc[df["Website"] == web, "Meta Keywords"] = has_metakeywords(soup)
    df.loc[df["Website"] == web, "Size KB"] = len(response.content) / 1024
    df.loc[df["Website"] == web, "JS Scripts"] = javascript_size(soup)
    df.loc[df["Website"] == web, "Words"] = num_words(soup)
    df.loc[df["Website"] == web, "Cookies"] = 'Set-Cookie' in response.headers
    
    

Proccessing:  http://bwin.es/
Proccessing:  https://www.independent.co.uk/sport
Proccessing:  https://www.surfer.com/
Error:  403  for  https://www.surfer.com/
Proccessing:  https://www.worldskate.org/
Proccessing:  https://theathletic.com/uk/
Proccessing:  https://www.mlb.com/
Proccessing:  https://worldrowing.com/
Proccessing:  https://www.mmafighting.com/
Proccessing:  https://www.sportico.com/
Proccessing:  https://www.cyclingnews.com/
Proccessing:  https://www.goal.com/
Proccessing:  https://www.businessinsider.es/
Proccessing:  https://racer.com/
Proccessing:  https://www.ufc.com/
Proccessing:  https://sports.yahoo.com/
Proccessing:  http://transfermarkt.es/
Proccessing:  https://www.livefutbol.com/
Proccessing:  https://geniussports.com/
Error:  403  for  https://geniussports.com/
Proccessing:  https://www.triathlete.com/
Proccessing:  https://frontofficesports.com/
Proccessing:  https://www.cricbuzz.com/
Proccessing:  https://www.world.rugby/
Proccessing:  https://www.cricket.c

In [19]:
df

,Website,Images,Gifs,Title Size,Links,Languages,h1,h2,h3,HTTPS,Paragraphs,Meta Description,Meta Keywords,JS Size,Size KB,Words,Cookies
0,http://bwin.es/,0,0,4,0,5,0,0,0,False,0,True,False,19,74.421875,34,True
1,https://www.independent.co.uk/sport,124,0,60,567,2,21,123,4,True,4,True,False,14,367.476562,1622,True
2,https://www.surfer.com/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.worldskate.org/,24,0,49,159,0,0,0,23,True,16,True,True,2,73.851562,384,True
4,https://theathletic.com/uk/,146,0,91,234,2,0,0,8,True,85,True,False,7,697.599609,973,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,https://www.usatoday.com/sports/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,https://www.ifsc-climbing.org/,47,0,67,141,2,1,8,0,True,7,True,False,47,479.004883,472,False
88,https://www.forbes.com/,0,0,6,183,2,0,14,3,True,4,True,True,14,402.945312,672,True
89,https://worldcurling.org/,16,0,20,70,5,1,5,9,True,21,False,False,4,85.443359,319,False
